In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.datasets.amass import AMASS
import clip
from src.datasets.get_dataset import get_datasets
from src.utils.get_model_and_data import get_model_and_data
import torch
import os
from src.utils.tensors import collate
from src.datasets.tools import condense_duplicates
import numpy as np
from src.utils.action_label_to_idx import action_label_to_idx
import src.utils.fixseed  # noqa
from src.utils.misc import load_model_wo_clip

In [ ]:
parameters = {
    'expname': 'exps',
    'folder': './exps/clip',
    'cuda': True,
    'device': torch.device(type='cuda', index=0),
    'batch_size': 80,
    'num_epochs': 500,
    'lr': 0.0002,
    'snapshot': 20,
    'dataset': 'babel',
    'datapath': './data/babel_30fps_db.pt',
    'num_frames': 60,
    'sampling': 'conseq',
    'sampling_step': 1,
    'pose_rep': 'rot6d',
    'max_len': -1,
    'min_len': -1,
    'num_seq_max': -1,
    'glob': True,
    'glob_rot': [3.141592653589793, 0, 0],
    'translation': True,
    'debug': False,
    'use_action_cat_as_text_labels': True,
    #'only_60_classes': True,
    'use_only_15_classes': False,
    'modelname': 'motionclip_transformer_rc_rcxyz_vel',
    'latent_dim': 768,
    'lambda_rc': 95.0,
    'lambda_rcxyz': 95.0,
    'lambda_vel': 95.0,
    'lambda_velxyz': 1.0,
    'jointstype': 'vertices',
    'vertstrans': False,
    'num_layers': 8,
    'activation': 'gelu',
    'clip_image_losses': ['cosine'],
    'clip_text_losses': ['cosine'],
    'clip_lambda_mse': 1.0,
    'clip_lambda_ce': 1.0,
    'clip_lambda_cosine': 1.0,
    'clip_training': '',
    'clip_layers': 12,
    'modeltype': 'motionclip',
    'archiname': 'transformer',
    'losses': ['rc', 'rcxyz', 'vel'],
    'lambdas': {'rc': 95.0, 'rcxyz': 95.0, 'vel': 95.0},
    'clip_lambdas': {'image': {'cosine': 1.0}, 'text': {'cosine': 1.0}},
    'num_classes': 1,
    'nfeats': 6,
    'njoints': 25,
    'outputxyz': True
}

In [ ]:
model, datasets = get_model_and_data(parameters, split="train")

print("Restore weights..")
checkpointpath = os.path.join('./exps/final_archi', 'checkpoint_cap_0207.pth.tar')
state_dict = torch.load(checkpointpath, map_location=parameters["device"])
load_model_wo_clip(model, state_dict)
model.eval()
dataset = datasets['train']

In [ ]:
freq = {}
for entry in dataset:
    if entry == None:
        continue
    for cat in entry['all_categories']:
        if cat in freq:
            freq[cat] += 1
        else:
            freq[cat] = 1

In [ ]:
lots = [(l[0], action_label_to_idx[l[0]]) for l in sorted(freq.items(), key=lambda x: -x[1])]
print([f'{l[0]}: {l[1]}' for l in lots if l[1] < 60])

In [ ]:
print(lots)

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.manifold import TSNE
import random
import matplotlib.pyplot as plt

transform = transforms.Compose([transforms.ToTensor()])
dataloader = DataLoader(dataset, batch_size=64, shuffle=False, collate_fn=collate)

# Set the model to evaluation mode
model.eval()

# Initialize lists to store embeddings and labels
embeddings_list = []
labels_list = []

# Extract embeddings from the model
with torch.no_grad():
    for batch in dataloader:
        if batch is None:
            continue
        batch = {key: val.to(torch.device('cuda')) if torch.is_tensor(val) and key != 'mask' and key != 'y'  else val for key, val in batch.items()}    
        outputs = model.encode_motion(batch)
        
        embeddings_list.extend(outputs.cpu().numpy())
        labels = [action_label_to_idx[random.choice(list(set(lbls) & set(lots))) if len(list(set(lbls) & set(lots))) else "take/pick something up"] for lbls in batch['all_categories']]
        labels_list.extend(labels)





In [ ]:
# Concatenate the lists to obtain the final embeddings and labels
embeddings = np.array([emb for (idx, emb) in enumerate(embeddings_list) if labels_list[idx] < 20])
labels = np.array([l for l in labels_list if l < 20])

In [ ]:
import joblib

In [ ]:
l2gen_mist = joblib.load('label_mapping_mistral.pt')
print(l2gen_mist.keys())

l2gengpt4 = joblib.load('label_mapping.pt')

In [ ]:
#print(l2gen_mist['run'])
print(l2gengpt4['grab body part'])

In [ ]:
print(l2gen)

In [ ]:
# ground_truth = list(action_label_to_idx.keys())[:60]
# #embeddings = model.encode_text([l2gen[label] for label in ground_truth[:60]])
# embeddings = np.array(model.encode_text(l2gen[label] for label in ground_truth[:60]]).cpu().numpy())
# labels = np.array(list(range(60)))

In [ ]:
import umap

In [ ]:
# Apply t-SNE to reduce the dimensionality of embeddings
tsne = umap.UMAP(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(embeddings)

In [ ]:
# Plot the t-SNE embeddings
plt.figure(figsize=(11, 10))
unique_labels = np.unique(labels)
label_names = {label: f"brosadasdasddas" for label in unique_labels}

# for i, label in enumerate(range(len(embeddings))):
#     plt.annotate('\n'.join(ground_truth[label].split(' ')), (embeddings_tsne[i, 0], embeddings_tsne[i, 1]), fontsize=10)
scatter = plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], c=labels,s=1, cmap="tab20")
plt.title('Learned latent space for poses', fontsize=18)
cbar = plt.colorbar(scatter, ticks=np.unique(range(20)))
cbar.set_ticklabels([lots[i] for i in np.unique(range(20))])
cbar.ax.tick_params(labelsize=18)
plt.savefig('memb_cap.png', bbox_inches = 'tight')
plt.show()

In [ ]:
train_dataset = get_datasets(parameters=parameters, split='train')['train']
#val_dataset = get_datasets(parameters=parameters, split='vald')['test']

In [ ]:
print(len(train_dataset))

In [ ]:
print(freq)

In [ ]:
import joblib
joblib.dump(freq, 'freq.pt')

In [ ]:
lots = [l[0] for l in sorted(freq.items(), key=lambda x: -x[1])]
lots = lots[:20]

In [ ]:
print(train_dataset.__getitem__(100))

In [ ]:
db = train_dataset.load_db()

In [ ]:
print(db['clip_text'][100])
print(db['action_cat'][100])

In [ ]:
l2gen = {}

for idx in range(len(db['action_cat'])):
    unique_cats = np.unique(db['action_cat'][idx])
    cats = []
    for cat in unique_cats:
        if cat not in action_label_to_idx.keys():
            continue
        cat_idx = action_label_to_idx[cat]
        if cat_idx >= 60:
            continue
        cats.append(cat)
    for (i, cat) in enumerate(cats):
        if cat in l2gen:
            l2gen[cat].append(db['clip_text'][idx][i])
        else:
            l2gen[cat] = [db['clip_text'][idx][i]]



In [ ]:
import joblib
joblib.dump(l2gen, 'label_mapping_multiple.pt')

In [ ]:
print(l2gen['walk'])

In [ ]:
print([key for key in list(action_label_to_idx.keys())[:60] if key not in l2gen.keys()] )

In [ ]:
paraphrasings = [
    "Elaborate on the bodily gestures of an individual executing the task: {}.",
    "Provide a detailed account of the physical movements exhibited by someone engaged in the action: {}.",
    "Explain thoroughly the body language of a person carrying out the specified activity: {}.",
    "Describe in depth the movements of a person's body while performing the action: {}.",
    "Articulate the intricate details of the physical gestures made by someone who is involved in {}.",
    "Offer a comprehensive depiction of the bodily actions of an individual executing the given task: {}.",
    "Narrate the specific body movements of a person undertaking the action: {}.",
    "Delve into the nuanced details of how a person's body moves during the performance of {}.",
    "Illustrate the intricate physical expressions of an individual carrying out the task: {}.",
    "Provide a detailed portrayal of the bodily maneuvers of someone participating in the action: {}.",
    "Elucidate the detailed body language exhibited by a person while engaged in the activity: {}.",
    "Examine closely the specific physical actions of a person performing the task: {}.",
    "Offer a thorough description of the body movements demonstrated by an individual involved in {}.",
    "Explain in detail the bodily expressions of a person executing the specified action: {}.",
    "Describe meticulously the physical actions undertaken by someone performing the task: {}.",
    "Present a detailed account of the body movements of an individual engaged in the action: {}.",
    "Articulate the specific physical gestures made by someone undertaking the task: {}.",
    "Narrate the intricate details of a person's body movements while performing the action: {}.",
    "Delve into the specifics of the bodily actions of an individual executing the given task: {}.",
    "Illustrate the nuanced body language displayed by a person participating in {}.",
    "Provide a comprehensive description of the bodily maneuvers of someone involved in the action: {}.",
    "Elucidate the detailed body language expressed by a person while carrying out the activity: {}.",
    "Examine closely the specific physical movements of a person performing {}.",
    "Offer a thorough account of the body gestures exhibited by an individual engaged in the action: {}.",
    "Explain in detail the physical actions of a person executing the specified task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the action: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the action: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the given task: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the task: {}.",
    "Examine closely the specific physical movements of a person performing the action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the task: {}.",
    "Explain in detail the bodily actions of a person executing the specified {}.",
    "Describe meticulously the body movements demonstrated by someone performing the action: {}.",
    "Present a detailed portrayal of the specific physical maneuvers of an individual involved in {}.",
    "Articulate the nuanced details of the bodily expressions displayed by someone participating in the action: {}.",
    "Narrate the intricate specifics of a person's physical movements while performing the task: {}.",
    "Delve into the specifics of the body language exhibited by an individual executing the given action: {}.",
    "Illustrate the comprehensive physical actions of a person participating in {}.",
    "Provide a detailed description of the bodily gestures displayed by someone involved in the task: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the specified action: {}.",
    "Examine closely the specific physical movements of a person performing the {}.",
    "Offer a thorough account of the body language exhibited by an individual engaged in the specified task: {}.",
    "Explain in detail the bodily actions of a person executing the given action: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the specified task: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the action: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the specified task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the given action: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the action: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in the specified task: {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the specified action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the given task: {}.",
    "Examine closely the specific physical movements of a person performing the specified action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the specified task: {}.",
    "Explain in detail the bodily actions of a person executing the given task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the given action: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the specified task: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the given action: {}.",
    "Narrate the intricate specifics of a person's body language while performing the specified task: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the given action: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in the given task: {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the given action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the specified task: {}.",
    "Examine closely the specific physical movements of a person performing the given action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the specified action: {}.",
    "Explain in detail the bodily actions of a person executing the specified task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the specified task: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the specified action: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the specified task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the specified action: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the specified action: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in the specified task: {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the specified action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the specified task: {}.",
    "Examine closely the specific physical movements of a person performing the specified action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the specified task: {}.",
    "Explain in detail the bodily actions of a person executing the specified task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the specified action: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the specified task: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the specified task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the specified task: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the specified action: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in the specified task: {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the specified action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the specified task: {}.",
    "Examine closely the specific physical movements of a person performing the specified action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the specified task: {}.",
    "Explain in detail the bodily actions of a person executing the specified task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the specified action: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the specified task: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the specified task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the specified task: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the specified action: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in the specified task: {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the specified action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the specified task: {}.",
    "Examine closely the specific physical movements of a person performing the specified action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the specified task: {}.",
    "Explain in detail the bodily actions of a person executing the specified task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the specified action: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the specified task: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the specified task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the specified task: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the specified action: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in the specified task: {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the specified action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the specified task: {}.",
    "Examine closely the specific physical movements of a person performing the specified action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the specified task: {}.",
    "Explain in detail the bodily actions of a person executing the specified task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the specified action: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the specified task: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the specified task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the specified task: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the specified action: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in the specified task: {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the specified action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the specified task: {}.",
    "Examine closely the specific physical movements of a person performing the specified action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the specified task: {}.",
    "Explain in detail the bodily actions of a person executing the specified task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the specified action: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the specified task: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the specified task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the specified task: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the specified action: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in the specified task: {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the specified action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the specified task: {}.",
    "Examine closely the specific physical movements of a person performing the specified action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the specified task: {}.",
    "Explain in detail the bodily actions of a person executing the specified task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the specified action: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the specified task: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the specified task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the specified task: {}.",
    "Delve into the specifics of the physical actions undertaken by an individual executing the specified action: {}.",
    "Illustrate the comprehensive body language exhibited by a person participating in the specified task: {}.",
    "Provide a detailed depiction of the bodily gestures of someone involved in the specified action: {}.",
    "Elucidate the nuanced body language expressed by a person while carrying out the specified task: {}.",
    "Examine closely the specific physical movements of a person performing the specified action: {}.",
    "Offer a thorough account of the body language displayed by an individual engaged in the specified task: {}.",
    "Explain in detail the bodily actions of a person executing the specified task: {}.",
    "Describe meticulously the body movements demonstrated by someone involved in the specified action: {}.",
    "Present a detailed portrayal of the specific physical expressions of an individual performing the specified task: {}.",
    "Articulate the nuanced details of the bodily maneuvers displayed by someone participating in the specified task: {}.",
    "Narrate the intricate specifics of a person's body language while performing the specified task: {}.",
     "Elaborate on the body language of an individual executing the following actions: {}",
    "Provide a detailed account of the physical gestures of a person engaged in the actions: {}",
    "Describe thoroughly the bodily movements of someone performing the actions: {}",
    "Illustrate in detail the way a person's body moves while undertaking the actions: {}",
    "Explain meticulously the body gestures of an individual executing the given actions: {}",
    "Detail the physical motions of a person who is actively involved in the actions: {}",
    "Break down the specific body movements of someone carrying out the actions: {}",
    "Offer an intricate portrayal of the bodily expressions of an individual performing the actions: {}",
    "Outline in detail the way a person's body responds while executing the actions: {}",
    "Provide an exhaustive account of the physical actions of someone engaged in: {}",
    "Articulate the nuanced body language of an individual performing the specified actions: {}",
    "Present a comprehensive depiction of the bodily gestures involved in the actions: {}",
    "Discuss in detail the body movements of a person in the process of performing: {}",
    "Provide an elaborate description of the physical movements of someone undertaking the actions: {}",
    "Examine closely the body language exhibited by an individual performing the actions: {}",
    "Detail the intricate physical gestures of a person actively involved in the actions: {}",
    "Illustrate thoroughly the body movements of someone executing the given actions: {}",
    "Explain in detail the specific physical motions of a person performing: {}",
    "Break down the intricate details of the bodily expressions of an individual engaged in: {}",
    "Offer a comprehensive analysis of the body language displayed while performing the actions: {}",
    "Provide a detailed breakdown of the physical actions of someone actively engaged in: {}",
    "Describe meticulously the bodily movements of a person executing the given actions: {}",
    "Articulate in detail the specific body language of an individual performing: {}",
    "Present a thorough examination of the bodily gestures involved in executing the actions: {}",
    "Discuss in detail the way a person's body moves while performing the specified actions: {}",
    "Provide an exhaustive portrayal of the physical movements of someone in the process of: {}",
    "Examine closely the body language exhibited by an individual actively performing: {}",
    "Detail the intricate physical gestures of a person involved in the actions: {}",
    "Illustrate thoroughly the specific body movements of someone executing: {}",
    "Explain in detail the nuanced physical motions of a person performing the specified actions: {}",
    "Break down the intricate details of the bodily expressions displayed while performing: {}",
    "Offer a comprehensive analysis of the body language observed in someone actively engaged in: {}",
    "Provide a detailed breakdown of the specific physical actions of an individual performing the given: {}",
    "Describe meticulously the body movements of a person actively involved in performing the actions: {}",
    "Articulate in detail the specific bodily gestures of an individual executing the actions: {}",
    "Present a thorough examination of the physical movements involved in performing the specified actions: {}",
    "Discuss in detail the way a person's body responds while actively engaged in: {}",
    "Provide an exhaustive portrayal of the body language displayed by someone in the process of performing: {}",
    "Examine closely the specific physical gestures of a person performing the given actions: {}",
    "Detail the nuanced body movements of someone executing the actions: {}",
    "Illustrate thoroughly the specific physical motions of a person actively involved in performing: {}",
    "Explain in detail the intricate details of the bodily expressions exhibited while carrying out: {}",
    "Break down the comprehensive analysis of the body language observed in an individual performing the actions: {}",
    "Offer a detailed breakdown of the specific physical actions of someone engaged in: {}",
    "Describe meticulously the body language displayed by a person performing the specified actions: {}",
    "Articulate in detail the specific body movements involved in executing the actions: {}",
    "Present a thorough examination of the bodily gestures displayed while actively engaged in performing: {}",
    "Discuss in detail the way a person's body moves during the process of: {}",
    "Provide an exhaustive portrayal of the physical movements observed in someone actively performing the actions: {}",
    "Examine closely the body language exhibited by an individual executing the given actions: {}",
    "Detail the intricate physical gestures of a person actively involved in performing: {}",
    "Illustrate thoroughly the specific body movements of someone performing the specified actions: {}",
    "Explain in detail the nuanced physical motions of a person undertaking the actions: {}",
    "Break down the intricate details of the bodily expressions exhibited while executing: {}",
    "Offer a comprehensive analysis of the body language observed in an individual performing the specified: {}",
    "Provide a detailed breakdown of the specific physical actions of a person engaged in performing the actions: {}",
    "Describe meticulously the body movements of an individual executing the actions: {}",
    "Articulate in detail the specific bodily gestures of someone actively involved in: {}",
    "Present a thorough examination of the physical movements displayed while performing the actions: {}",
    "Discuss in detail the way a person's body responds while executing the specified actions: {}",
    "Provide an exhaustive portrayal of the body language observed in someone performing the given actions: {}",
    "Examine closely the specific physical gestures of a person performing the actions: {}",
    "Detail the nuanced body movements of someone executing the specified actions: {}",
    "Illustrate thoroughly the specific physical motions of a person actively involved in: {}",
    "Explain in detail the intricate details of the bodily expressions exhibited during: {}",
    "Break down the comprehensive analysis of the body language displayed by an individual performing: {}",
    "Offer a detailed breakdown of the specific physical actions of someone executing the actions: {}",
    "Describe meticulously the body language of a person performing the actions: {}",
    "Articulate in detail the specific body movements of an individual actively involved in: {}",
    "Present a thorough examination of the bodily gestures exhibited while executing the specified actions: {}",
    "Discuss in detail the way a person's body moves during the process of performing: {}",
    "Provide an exhaustive portrayal of the physical movements observed in someone actively performing the given actions: {}",
    "Examine closely the body language displayed by an individual executing the actions: {}",
    "Detail the intricate physical gestures of a person actively involved in performing the specified actions: {}",
    "Illustrate thoroughly the specific body movements of someone performing the given actions: {}",
    "Explain in detail the nuanced physical motions of a person undertaking the specified actions: {}",
    "Break down the intricate details of the bodily expressions exhibited while performing the actions: {}",
    "Offer a comprehensive analysis of the body language observed in an individual performing the specified actions: {}",
    "Provide a detailed breakdown of the specific physical actions of a person engaged in performing: {}",
    "Describe meticulously the body movements of an individual executing the given actions: {}",
    "Articulate in detail the specific bodily gestures of someone actively involved in performing the actions: {}",
    "Present a thorough examination of the physical movements displayed while executing the given actions: {}",
    "Discuss in detail the way a person's body responds while actively engaged in performing the specified actions: {}",
    "Provide an exhaustive portrayal of the body language observed in someone performing the actions: {}",
    "Examine closely the specific physical gestures of a person performing the specified actions: {}",
    "Detail the nuanced body movements of someone executing the given actions: {}",
    "Illustrate thoroughly the specific physical motions of a person actively involved in performing the specified actions: {}",
    "Explain in detail the intricate details of the bodily expressions exhibited while undertaking the actions: {}",
    "Break down the comprehensive analysis of the body language displayed by an individual performing the given actions: {}",
    "Offer a detailed breakdown of the specific physical actions of someone executing the specified actions: {}",
    "Describe meticulously the body language of a person performing the specified actions: {}",
    "Articulate in detail the specific body movements of an individual actively involved in performing the given actions: {}",
    "Present a thorough examination of the bodily gestures exhibited while executing the actions: {}",
    "Discuss in detail the way a person's body moves during the process of performing the specified actions: {}",
    "Provide an exhaustive portrayal of the physical movements observed in someone actively performing the specified actions: {}",
    "Examine closely the body language displayed by an individual executing the specified actions: {}",
    "Detail the intricate physical gestures of a person actively involved in performing the given actions: {}",
    "Illustrate thoroughly the specific body movements of someone performing the specified actions: {}",
    "Explain in detail the nuanced physical motions of a person undertaking the given actions: {}",
    "Break down the intricate details of the bodily expressions exhibited while performing the specified actions: {}",
    "Offer a comprehensive analysis of the body language observed in an individual performing the actions: {}",
    "Provide a detailed breakdown of the specific physical actions of a person engaged in performing the given actions: {}",
    "Describe meticulously the body movements of an individual executing the specified actions: {}",
    "Articulate in detail the specific bodily gestures of someone actively involved in performing the given actions: {}",
    "Present a thorough examination of the physical movements displayed while executing the specified actions: {}",
    "Discuss in detail the way a person's body responds while actively engaged in performing the given actions: {}",
    "Provide an exhaustive portrayal of the body language observed in someone performing the specified actions: {}",
    "Examine closely the specific physical gestures of a person performing the given actions: {}",
    "Detail the nuanced body movements of someone executing the specified actions: {}",
    "Illustrate thoroughly the specific physical motions of a person actively involved in performing the given actions: {}",
    "Explain in detail the intricate details of the bodily expressions exhibited while undertaking the specified actions: {}",
    "Break down the comprehensive analysis of the body language displayed by an individual performing the given actions: {}",
    "Offer a detailed breakdown of the specific physical actions of someone executing the given actions: {}",
    "Describe meticulously the body language of a person performing the given actions: {}",
    "Articulate in detail the specific body movements of an individual actively involved in performing the specified actions: {}",
    "Present a thorough examination of the bodily gestures exhibited while executing the given actions: {}",
    "Discuss in detail the way a person's body moves during the process of performing the given actions: {}",
    "Provide an exhaustive portrayal of the physical movements observed in someone actively performing the given actions: {}",
    "Examine closely the body language displayed by an individual executing the given actions: {}",
    "Detail the intricate physical gestures of a person actively involved in performing the specified actions: {}",
    "Illustrate thoroughly the specific body movements of someone performing the given actions: {}",
    "Explain in detail the nuanced physical motions of a person undertaking the given actions: {}",
    "Break down the intricate details of the bodily expressions exhibited while performing the given actions: {}",
    "Offer a comprehensive analysis of the body language observed in an individual performing the specified actions: {}",
       "Explain the bodily gestures of someone executing the tasks: {}",
    "Elaborate on the physical actions of an individual engaged in the activities: {}",
    "Illustrate the detailed movements of a person carrying out the actions: {}",
    "Provide a thorough description of the body language of someone performing the actions: {}",
    "Describe in depth the gestures and motions of a person who is undertaking the actions: {}",
    "Give a detailed account of the physical expressions of an individual executing the tasks: {}",
    "Offer a comprehensive explanation of the bodily maneuvers of someone involved in the actions: {}",
    "Break down the intricate body movements of a person performing the specified actions: {}",
    "Detail the precise physical actions of an individual in the process of performing the actions: {}",
    "Outline the nuanced body language of someone actively engaged in the actions: {}",
    "Present an elaborate description of the person's body movements while performing the actions: {}",
    "Provide a comprehensive overview of the bodily actions of a person carrying out the specified tasks: {}",
    "Articulate the specific body language exhibited by an individual performing the actions: {}",
    "Explain in detail the bodily expressions of someone executing the actions: {}",
    "Break down the intricate details of the physical movements of a person performing the actions: {}",
    "Describe the detailed body language of someone actively participating in the actions: {}",
    "Elucidate the nuanced bodily expressions of an individual undertaking the actions: {}",
    "Provide a detailed analysis of the person's physical gestures while performing the actions: {}",
    "Offer a thorough examination of the body movements of someone engaged in the actions: {}",
    "Illustrate the detailed physical actions of a person in the process of performing the specified tasks: {}",
    "Outline the specific body language displayed by an individual performing the actions: {}",
    "Detail the intricate bodily maneuvers of someone executing the actions: {}",
    "Explain the specific physical expressions of a person actively involved in the actions: {}",
    "Provide a comprehensive breakdown of the body movements of someone carrying out the actions: {}",
    "Describe in detail the bodily gestures and motions of an individual performing the specified actions: {}",
    "Elaborate on the precise body language exhibited by a person undertaking the actions: {}",
    "Present a detailed account of the physical actions of someone executing the specified tasks: {}",
    "Articulate the nuanced body movements of an individual actively engaged in the actions: {}",
    "Offer a comprehensive explanation of the bodily expressions displayed by a person performing the actions: {}",
    "Provide a detailed analysis of the specific body language of someone involved in the actions: {}",
    "Break down the intricate details of the physical gestures and motions of a person performing the actions: {}",
    "Describe the detailed body movements of someone actively participating in the actions: {}",
    "Elucidate the nuanced bodily actions of an individual undertaking the specified tasks: {}",
    "Outline the specific physical expressions displayed by a person performing the actions: {}",
    "Detail the intricate body language of someone executing the actions: {}",
    "Explain the specific bodily maneuvers of a person actively involved in the specified actions: {}",
    "Provide a comprehensive breakdown of the detailed physical actions of someone carrying out the tasks: {}",
    "Illustrate the nuanced body movements of an individual performing the actions: {}",
    "Present a detailed account of the specific body language exhibited by a person undertaking the specified actions: {}",
    "Articulate the precise bodily expressions displayed by someone executing the actions: {}",
    "Offer a comprehensive explanation of the detailed physical gestures and motions of a person performing the actions: {}",
    "Provide a thorough examination of the specific body movements of someone actively engaged in the actions: {}",
    "Break down the intricate details of the bodily actions of a person performing the actions: {}",
    "Provide a detailed description of the bodily movements of someone engaged in the actions: {}",
    "Explain thoroughly the body movements of an individual carrying out the specified actions: {}",
    "Elaborate on the physical gestures of a person executing the given actions: {}",
    "Describe in intricate detail the movements of the body of an individual who is performing the following actions: {}",
    "Illustrate the specific body motions of someone undertaking the actions: {}",
    "Give a detailed account of the physical movements exhibited by a person executing the specified actions: {}",
    "Break down the bodily gestures of an individual involved in the actions: {}",
    "Provide an elaborate depiction of the body movements of someone carrying out the given actions: {}",
    "Articulate the nuances of the physical motions displayed by a person performing the actions: {}",
    "Delve into the intricacies of the bodily movements of an individual engaged in the specified actions: {}",
    "Offer a detailed portrayal of the body language of someone performing the actions: {}",
    "Examine closely the physical actions and movements of a person involved in the specified activities: {}",
    "Present a comprehensive analysis of the bodily expressions of an individual executing the actions: {}",
    "Go into detail about the body movements of a person who is actively involved in the specified actions: {}",
    "Provide a thorough breakdown of the physical gestures exhibited by someone performing the actions: {}",
    "Describe the intricate details of a person's body language while engaging in the specified actions: {}",
    "Elaborate on the specific body movements of an individual carrying out the given actions: {}",
    "Illustrate the detailed physical actions of a person involved in the specified activities: {}",
    "Articulate the nuances of the body language exhibited by someone performing the actions: {}",
    "Present a detailed account of the bodily movements of an individual executing the given actions: {}",
    "Explain in detail the physical gestures of a person actively participating in the specified actions: {}",
    "Break down the specific body motions of someone undertaking the given actions: {}",
    "Provide a comprehensive analysis of the body language displayed by a person performing the actions: {}",
    "Delve into the intricacies of the bodily movements of an individual engaged in the specified activities: {}",
    "Offer a detailed description of the physical expressions of someone carrying out the actions: {}",
    "Examine closely the detailed body movements of a person involved in the specified actions: {}",
    "Present a thorough breakdown of the physical gestures exhibited by someone performing the actions: {}",
    "Describe in intricate detail the body language of an individual actively participating in the specified actions: {}",
    "Illustrate the specific body movements of a person carrying out the given actions: {}",
    "Articulate the nuanced details of the bodily actions displayed by someone executing the actions: {}",
    "Provide a comprehensive account of the body language exhibited by a person performing the specified actions: {}",
    "Explain in detail the physical movements of someone actively participating in the specified activities: {}",
    "Break down the specific body motions of an individual undertaking the given actions: {}",
    "Delve into the intricacies of the bodily expressions of a person involved in the actions: {}",
    "Offer a detailed portrayal of the physical gestures exhibited by someone performing the specified actions: {}",
    "Examine closely the detailed body movements of an individual carrying out the given actions: {}",
    "Present a thorough breakdown of the specific body motions displayed by someone executing the actions: {}",
    "Describe in intricate detail the nuances of the body language of a person actively participating in the specified actions: {}",
    "Illustrate the specific body movements of someone performing the given actions: {}",
    "Articulate the detailed physical actions displayed by a person engaged in the specified activities: {}",
    "Provide a comprehensive analysis of the body language exhibited by an individual actively participating in the specified actions: {}",
    "Explain in detail the specific bodily movements of someone undertaking the actions: {}",
    "Break down the intricacies of the physical gestures exhibited by a person performing the specified actions: {}",
    "Delve into the detailed body movements of an individual actively participating in the given actions: {}",
    "Offer a thorough breakdown of the specific body motions displayed by someone executing the actions: {}",
    "Examine closely the nuanced details of the bodily expressions of a person involved in the specified activities: {}",
    "Present a detailed account of the physical gestures exhibited by someone carrying out the specified actions: {}",
    "Describe in intricate detail the body language of an individual performing the given actions: {}",
    "Illustrate the specific body movements of a person executing the specified actions: {}",
    "Articulate the detailed physical actions displayed by someone actively participating in the given activities: {}",
    "Provide a comprehensive analysis of the body language exhibited by an individual involved in the specified actions: {}",
    "Explain in detail the specific bodily movements of someone carrying out the actions: {}",
    "Break down the intricacies of the physical gestures exhibited by a person performing the given actions: {}",
    "Delve into the detailed body movements of an individual actively participating in the specified activities: {}",
    "Offer a thorough breakdown of the specific body motions displayed by someone executing the specified actions: {}",
    "Examine closely the nuanced details of the bodily expressions of a person involved in the given actions: {}",
    "Present a detailed account of the physical gestures exhibited by someone carrying out the actions: {}",
    "Describe in intricate detail the body language of an individual performing the specified actions: {}",
    "Illustrate the specific body movements of a person executing the given actions: {}",
    "Articulate the detailed physical actions displayed by someone actively participating in the specified activities: {}",
]

In [ ]:
print(len(paraphrasings))

In [ ]:
l2gen = {}

In [ ]:
print(db.keys())

In [ ]:
import os 
from vllm import LLM, SamplingParams
import time

llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2", max_model_len=8000)
sampling_params = SamplingParams(temperature=0.5, max_tokens=512)

In [ ]:
output llm.generate(prompts, sampling_params)

In [ ]:
prompts = []
for label in list(action_label_to_idx.keys())[:60]:
    for i in range(377):
        prompts.append(paraphrasings[i].format(f"'{label}.'"))



In [ ]:
outputs = llm.generate(prompts, sampling_params)

In [ ]:
print(len(outputs))

In [ ]:
l2gen = {}
for label in list(action_label_to_idx.keys())[:60]:
    l2gen[label] = []

In [ ]:
start = 0
for (idx, label) in enumerate(list(action_label_to_idx.keys())[:60]):
    texts = [output.outputs[0].text for output in outputs[start: start + 377]]
    l2gen[label].extend(texts)
    start += 377

In [ ]:
print(l2gen['sideways movement'][370])

In [ ]:
import random
prompts = []
for idx in range(len(db['action_cat'])):
    unique_cats = np.unique(db['action_cat'][idx])
    for cat in unique_cats:
        if cat not in action_label_to_idx:
            continue
        cat_idx = action_label_to_idx[cat]
        if cat_idx >= 60:
            continue
        prompts.append((idx, cat))

p = [random.choice(paraphrasings).format(f"'{prompt[1]}'")
     for prompt in prompts]

outputs = llm.generate(p, sampling_params)
results = [(output.request_id, output.outputs[0].text) for output in outputs]
    

In [ ]:
start = 0
#results = sorted(results, key=lambda x: x[0])
tresults = [result[1] for result in results]
db['clip_text'] = []
print(prompts[100])
print(results[100])
for idx in range(len(db['action_cat'])):
    unique_cats = np.unique(db['action_cat'][idx])
    count = 0
    for cat in unique_cats:
        if cat not in action_label_to_idx:
            continue
        cat_idx = action_label_to_idx[cat]
        if cat_idx >= 60:
            continue
        if idx == 100:
            print('here')
            print(start)
            print(tresults[start:start + count + 1])
        count += 1
    db['clip_text'].append(tresults[start:start + count])
    start += count
    

In [ ]:
p1 = parameters
p1['datapath'] = 'babel_with_gen_db.pt'
train_dataset = get_datasets(parameters=p1, split='train')['train']
val_dataset = get_datasets(parameters=p1, split='vald')['test']

In [ ]:
print(train_dataset.__getitem__(100))

In [ ]:
[[1, 2, 3]] * 3

In [ ]:
print(db['clip_text'][1500])
print(db['action_cat'][1500])

In [ ]:
train_dataset.db = db

In [ ]:
print(train_datset.__getitem__(100))

In [ ]:
print(db['thetas'][0])

In [ ]:
import joblib

In [ ]:
joblib.dump(db, 'babel_with_gen_train.pt')
joblib.dump(val_dataset.db, 'babel_with_gen_vald.pt')

In [ ]:
print(len(db['action_cat']))

In [ ]:
print(db['text_raw_labels'][0])

In [ ]:
import torch

# Generate a sample tensor
data = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])

# Define quantiles
quantiles = torch.tensor([0.25, 0.5, 0.75])

# Compute quantiles
quantile_values = torch.quantile(data, quantiles)

# Perform quantile transform
quantile_transformed_data = torch.searchsorted(quantile_values, data.float()) / float(len(quantiles))

print("Original data:", data)
print("Quantile-transformed data:", quantile_transformed_data)

In [ ]:
import joblib
text_descriptions = joblib.load('./data/multiple_captions.pt')

In [ ]:
print(text_descriptions.keys())

In [ ]:
from angle_emb import AnglE

In [ ]:
import torch

# Example tensor matrix
matrix = torch.tensor([[1.0, 2.0, 3.0],
                      [0.0, 0.0, 0.0],
                      [4.0, 5.0, 6.0],
                      [0.0, 1.0, 0.0]])

# Create a mask for non-zero rows
nonzero_rows_mask = (matrix != 0).any(dim=1)

# Filter out rows where all elements are zero
filtered_matrix = matrix[nonzero_rows_mask]

print("Original Matrix:")
print(matrix)
print("\nFiltered Matrix (non-zero rows only):")
print(filtered_matrix)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
print(text_descriptions['step'][0])
print(text_descriptions['leg movements'][0])
sentences = [text_descriptions['leg movements'][50], text_descriptions['leg movements'][0]]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
# sentence_embedfeatures / features.norm(dim=-1, keepdim=True)dings[1] = F.normalize(sentence_embeddings, p=2, dim=1)
sentence_embeddings = sentence_embeddings / sentence_embeddings.norm(dim=-1, keepdim=True)
print("Sentence embeddings:")
print(sentence_embeddings[0] @ sentence_embeddings[1].T)

In [ ]:
text_encoder = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

In [ ]:
from src.utils.action_label_to_idx import action_label_to_idx
import torch.nn.functional as F
minim = 300
label1 = text_encoder.encode("going out with the canine", to_numpy=False)
label1 = F.normalize(label1, p=2, dim=1)
label2 = text_encoder.encode("walking the dog", to_numpy=False)
label2 = F.normalize(label2, p=2, dim=1)
print(F.cosine_similarity(label1, label2))
print(label1 @  label2.T)
# for action in action_label_to_idx.keys():
#     label1 = text_encoder.encode(action, to_numpy=False)
#     label2 = text_encoder.encode(action, to_numpy=False)
#     minim = min(minim, label1 @  label2.T)
# print(minim)

In [ ]:
print(train_dataset.__getitem__(100))

In [ ]:
db = train_dataset.load_db()

In [ ]:
from src.datasets.tools import condense_duplicates

print(db.keys())
print(len(db['clip_images']))
print(db['text_raw_labels'][1])

labels = db['text_raw_labels'][1]
labels.append('lie down on stomach')
proc_labels = condense_duplicates(labels)
print(proc_labels)

In [ ]:
print(len(train_dataset))
print(train_dataset.datapath)
db = train_dataset.load_db()


In [ ]:
print(db.keys())
print(condense_duplicates(db['action_cat'][100]))

In [ ]:
import copy
import joblib

def gen_simple_dataset(db, split='train'):
    generated_db = copy.deepcopy(db)
    generated_db['clip_text'] = copy.deepcopy(db['text_proc_labels'])
    curated_idx = []
    for (idx, video_labels) in enumerate(db['text_raw_labels']):
        condensed_labels = condense_duplicates(video_labels)
        action = " and ".join(condensed_labels)
        generated_db['clip_text'][idx] = action
    
    print(generated_db['clip_text'][0])
    joblib.dump(generated_db, f'./data/babel/babel_30fps_{split}.pt')

#gen_simple_dataset(db)

In [ ]:
def gen_captions(llm, sampling_params):
    actions = list(action_label_to_idx.keys())[:60]
    prompts = ['[INST] Describe a person’s body movements who is performing the actions {} in detail [/INST]'
               .format(action) for action in actions]
    outputs = llm.generate(prompts, sampling_params)
    initial_results = [output.outputs[0].text for output in outputs]
    
    captions_mapping = {}
    for (idx, action) in enumerate(actions):
        captions_mapping[action] = [initial_results[idx]]
    prompts = []
    for result in initial_results:
        prompts.extend(['[INST] Paraphrase the following description: {} [/INST]'.format(result)] * 1000)
        
    outputs = llm.generate(prompts, sampling_params)
    for (idx, output) in enumerate(outputs):
        captions_mapping[actions[idx // 1000]].append(output.outputs[0].text)
        
    return captions_mapping
    
def gen_llm_dataset(db, llm, sampling_params, split='train'):
    generated_db = {}
    for key in db.keys():
        generated_db[key] = []
    curated_idx = []
    
    
    for (idx, vide_cat) in enumerate(db['action_cat'][:1]):
        unique_cats = np.unique(vide_cat)
        all_valid_cats = []
        for multi_cats in unique_cats:
            for cat in multi_cats.split(","):
                if cat not in action_label_to_idx:
                    continue
                
                cat_idx = action_label_to_idx[cat]
                if cat_idx >= 60:
                    continue
                all_valid_cats.extend([cat])

        if len(all_valid_cats) == 0:  # No valid category available
            continue
        
        for key in generated_db.keys():
            generated_db[key].append(db[key][idx])
        
        choosen_cat = np.random.choice(all_valid_cats, size=1)[0]
        # condensed_labels = condense_duplicates(video_labels)
        # action = " and ".join(condensed_labels)
        actions.append(choosen_cat)
    print(actions)
    captions = gen_captions(actions, llm, sampling_params)
    generated_db['clip_text'] = []
    for (idx, video_labels) in enumerate(generated_db['action_cat']):
        generated_db['clip_text'].append(captions[idx])
    
    print(len(generated_db['clip_text']))
    print(generated_db['clip_text'][0])
    joblib.dump(generated_db, f'./data/babel_llm_multiple/babel_30fps_{split}.pt')

In [ ]:
print(train_dataset._clip_texts[100])

In [ ]:
import os 
from vllm import LLM, SamplingParams
import time

llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1")
sampling_params = SamplingParams(temperature=0.7, max_tokens=512)

In [ ]:
captions = gen_captions(llm, sampling_params)

In [ ]:
print(captions['touch object'][:5])
joblib.dump(captions, f'./data/multiple_captions.pt')
1/0

In [ ]:
from src.utils.action_label_to_idx import action_label_to_idx
import joblib
def gen_groundtruth():
    
    action_text_labels = list(action_label_to_idx.keys())
    action_text_labels.sort(key=lambda x: action_label_to_idx[x])
    action_text_labels = action_text_labels[:60]
    print(action_text_labels)
    prompts = ['[INST] Describe a person’s body movements who is performing the actions {} in detail [/INST]'
               .format(action) for action in action_text_labels]
    outputs = llm.generate(prompts, sampling_params)
    return [{'generated': output.outputs[0].text, 'orig': action_text_labels[idx]}  for (idx, output) in enumerate(outputs)]

#joblib.dump(gen_groundtruth(), f'./data/babel_llm/grountruth.pt')

In [ ]:
 gen_llm_dataset(val_dataset.load_db(), llm, sampling_params, split='vald')

In [ ]:
gen_val_dataset = AMASS(clip_preprocess=clip_preprocess, datapath="./data/babel_llm_1/babel_30fps_db.pt")
print(len(gen_val_dataset))
print(gen_val_dataset.__getitem__(0)['all_categories'])

In [ ]:
count = 0
for entry in gen_val_dataset:
    if 'all_categories'  not in entry:
        count += 1
print(count)

In [ ]:
simple_val_dataset = AMASS(clip_preprocess=clip_preprocess, datapath="./data/babel/babel_30fps_db.pt")
print(len(simple_val_dataset))

In [ ]:
gen_train_dataset = AMASS(clip_preprocess=clip_preprocess, datapath="./data/babel_llm_1/babel_30fps_db.pt", split="train")

In [ ]:
db_val = gen_val_dataset.load_db()

In [ ]:
print(db_val.keys())

In [ ]:
print(gen_val_dataset._clip_texts[1])

In [ ]:
print(len(gen_val_dataset._clip_texts))
print(gen_val_dataset.__getitem__(5000)['clip_text'])
print(simple_val_dataset.__getitem__(5000)['clip_text'])

In [ ]:
prompts = ['[INST] Describe a person’s body movements who is performing the actions {} in detail [/INST]'.format('sway')]

In [ ]:
outputs = llm.generate(prompts, sampling_params)
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(generated_text)